In [1]:
!pip install accelerate -U
!pip install transformers soundfile datasets jiwer gdown pyctcdecode kenlm

  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-linux_x86_64.whl size=3184426 sha256=3ed558674383bc3a773998ceb3f3effaf27844fb287fda0c4c7f9bd56968e546
  Stored in directory: /root/.cache/pip/wheels/fd/80/e0/18f4148e863fb137bd87e21ee2bf423b81b3ed6989dab95135
Successfully built kenlm


In [2]:
!mkdir ./dataset
import gdown
def drive_download(idx, output):
    url = 'https://drive.google.com/uc?id=' + idx
    gdown.download(url, output, quiet=False)
drive_download("1ZBL3h6bHMmd8MIUNXqg72PucUkC9ZSWJ", "./dataset/train_data.zip")
# drive_download("1ZepptsTrVSjQEx-dpBBmQ2b7xYFLn_64", "./dataset/public_test.zip")
# drive_download("1K_07kix1OgBGO2FNPh-Lxqr1yLbtqFYt", "./dataset/train.jsonl")

drive_download("186Tv-dPED5QiIJy4sRvlsNsvYxLpXfWX", "./vn_base_vocab.json")
drive_download("1K-oNwBu2svshAkmifU9wISKPMvKgeKy4", "./train_20230909.jsonl")
# drive_download("1-sKIn6-MMt1S5wbNuK2etmJZYztTlS8b", "./4gram_correct.arpa")

Downloading...
From: https://drive.google.com/uc?id=1ZBL3h6bHMmd8MIUNXqg72PucUkC9ZSWJ
To: /content/dataset/train_data.zip
100%|██████████| 733M/733M [00:08<00:00, 82.8MB/s]


In [ ]:
# !unzip ./dataset/public_test.zip -d ./dataset/test
!unzip ./dataset/train_data.zip -d ./dataset/train

In [9]:
import torch, json, utils, os
import numpy as np
from functools import partial
from datasets import load_metric
from dataset import Wav2VecDataset
from torch.utils.data import DataLoader
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, TrainingArguments, Trainer, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor

In [5]:
def custom_collate(processor, batch):
    processed_batch = [
        processor(i["input_values"], text=i["label"], sampling_rate=16000) for i in batch
    ]
    input_features = [{"input_values": i.input_values[0]} for i in processed_batch]
    input_features = processor.feature_extractor.pad(input_features, padding=True, return_tensors="pt")
    if batch[0]["label"] is None:
        return input_features
    label_features = [{"input_ids": i.labels} for i in processed_batch]
    label_features = processor.tokenizer.pad(
        label_features,
        padding=True,
        return_tensors="pt"
    )
    label_features = label_features["input_ids"].masked_fill(label_features.attention_mask.ne(1), -100)
    input_features["labels"] = label_features
    return input_features

In [6]:
def train_test_split(root_path, notation_file, test_size=0.3):
    notations = utils.load_annotation(notation_file)
    dataset = Wav2VecDataset(root_path, [i["file"] for i in notations], notations)
    if test_size == 0:
      return dataset, dataset
    N = len(dataset)
    print(f"Len dataset: {N}")
    train_size = int(N * (1-test_size))
    train_set, valid_set = torch.utils.data.random_split(dataset, [train_size, N-train_size])
    return train_set, valid_set

In [8]:
train_ds, valid_ds = train_test_split("./dataset/train/Train/", "./train_20230909.jsonl", test_size=0.3)
len(train_ds), len(valid_ds)

Len dataset: 7490


(5243, 2247)

In [10]:
tokenizer = Wav2Vec2CTCTokenizer("./vn_base_vocab.json")
extractor = Wav2Vec2FeatureExtractor.from_pretrained("nguyenvulebinh/wav2vec2-large-vi-vlsp2020")
processor = Wav2Vec2Processor(tokenizer=tokenizer, feature_extractor=extractor)

In [16]:
model = Wav2Vec2ForCTC.from_pretrained(
    "nguyenvulebinh/wav2vec2-large-vi-vlsp2020",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    # vocab_size=len(processor.tokenizer)
)
model.config.vocab_size = tokenizer.vocab_size
model.config.lambda_intermediate = 0.25
model.config.n_intermediate_layers = 8
# 768 - base model
model.lm_head = torch.nn.Linear(in_features=1024, out_features=tokenizer.vocab_size, bias=True)

model.freeze_feature_encoder()
model.gradient_checkpointing_enable()


Some weights of the model checkpoint at nguyenvulebinh/wav2vec2-large-vi-vlsp2020 were not used when initializing Wav2Vec2ForCTC: ['feature_transform.bn2.num_batches_tracked', 'feature_transform.bn3.running_mean', 'feature_transform.bn3.num_batches_tracked', 'feature_transform.bn3.weight', 'feature_transform.linear2.bias', 'feature_transform.linear1.weight', 'feature_transform.bn2.bias', 'feature_transform.bn3.running_var', 'feature_transform.bn2.running_mean', 'feature_transform.bn1.running_mean', 'feature_transform.linear3.bias', 'feature_transform.linear2.weight', 'feature_transform.linear3.weight', 'feature_transform.linear1.bias', 'feature_transform.bn3.bias', 'feature_transform.bn2.running_var', 'feature_transform.bn2.weight', 'feature_transform.bn1.weight', 'feature_transform.bn1.num_batches_tracked', 'feature_transform.bn1.bias', 'feature_transform.bn1.running_var']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task 

In [ ]:
from model import apply_gic_intermediate, Wav2Vec2EncoderLayerStableLayerNormOptimal
model = apply_gic_intermediate(model, 8)
model

In [24]:
class CTCTrainer(Trainer):

    def __init__(self, *args, **kwargs):
        super(CTCTrainer, self).__init__(*args, **kwargs)

    def param_sharing(self, model):
        params = model.lm_head.state_dict()
        for layer in model.wav2vec2.encoder.layers:
            if isinstance(layer, Wav2Vec2EncoderLayerStableLayerNormOptimal):
                layer.wrap_gic.classify_head.load_state_dict(params)
                if hasattr(layer.wrap_gic, "log_probs"):
                    del layer.wrap_gic.log_probs
                    # layer.wrap_gic.log_probs = None
        return model

    def _ctc_loss_calc(self, log_probs, model, inputs):
        labels = inputs["labels"]
        input_values = inputs["input_values"]
        attention_mask = torch.ones_like(input_values, dtype=torch.long) if "attention_mask" not in inputs else inputs["attention_mask"]
        if labels.max() >= model.config.vocab_size:
            raise ValueError(f"Label values must be <= vocab_size: {model.config.vocab_size}")

        attention_mask = (attention_mask)
        input_lengths = model._get_feat_extract_output_lengths(attention_mask.sum(-1)).to(torch.long)

        labels_mask = labels >= 0
        target_lengths = labels_mask.sum(-1)
        flattened_targets = labels.masked_select(labels_mask)

        # ctc_loss doesn't support fp16
        # log_probs = nn.functional.log_softmax(logits, dim=-1, dtype=torch.float32).transpose(0, 1)

        with torch.backends.cudnn.flags(enabled=False):
            loss = torch.nn.functional.ctc_loss(
                log_probs,
                flattened_targets,
                input_lengths,
                target_lengths,
                blank=model.config.pad_token_id,
                reduction=model.config.ctc_loss_reduction,
                zero_infinity=model.config.ctc_zero_infinity,
            )
        return loss

    def inter_loss_calc(self, model, inputs):
        inters_loss = torch.Tensor().requires_grad_(True).to(self.args.device)
        for layer in model.wav2vec2.encoder.layers:
            if isinstance(layer, Wav2Vec2EncoderLayerStableLayerNormOptimal) and hasattr(layer.wrap_gic, "log_probs"):
                # print(hasattr(layer.wrap_gic, "log_probs"))
                log_probs = getattr(layer.wrap_gic, "log_probs")
                # log_probs = layer.wrap_gic.log_probs
                loss = self._ctc_loss_calc(log_probs, model, inputs)
                inters_loss = torch.cat([inters_loss, loss.view(1,)], dim=0)
        return inters_loss.mean()

    def total_loss_calc(self, lamb, loss, inter_loss):
        return (1.0 - lamb) * loss + lamb * inter_loss

    def training_step(self, model, inputs):
        model = self.param_sharing(model)
        model.train()
        inputs = self._prepare_inputs(inputs)
        if self.args.fp16:
            with torch.cuda.amp.autocast():
                outputs = model(**inputs)
                # loss = self.compute_loss(model, inputs)
                loss = outputs.loss
                inter_loss = self.inter_loss_calc(model, inputs)
        else:
            outputs = model(**inputs)
            # loss = self.compute_loss(model, inputs)
            loss = outputs.loss
            inter_loss = self.inter_loss_calc(model, inputs)
        # self.log({"Inter loss": inter_loss.detach()})
        all_loss = self.total_loss_calc(model.config.lambda_intermediate, loss, inter_loss)

        if self.args.n_gpu > 1:
            all_loss = all_loss.mean()

        if self.args.gradient_accumulation_steps > 1:
            all_loss = all_loss / self.args.gradient_accumulation_steps

        if self.do_grad_scaling:
            self.scaler.scale(loss).backward()
        else:
            self.accelerator.backward(loss)

        return all_loss.detach()

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        self.param_sharing(model)
        model.eval()
        inputs = self._prepare_inputs(inputs)
        with torch.no_grad():
            if self.args.fp16:
                with torch.cuda.amp.autocast():
                    outputs = model(**inputs)
                    loss = outputs.loss
                    # loss, outputs = self.compute_loss(model, inputs, return_outputs=True)
                    inter_loss = self.inter_loss_calc(model, inputs)
                    all_loss = self.total_loss_calc(model.config.lambda_intermediate, loss, inter_loss)
            else:
                loss, outputs = self.compute_loss(model, inputs, return_outputs=True)
                inter_loss = self.inter_loss_calc(model, inputs)
                all_loss = self.total_loss_calc(model.config.lambda_intermediate, loss, inter_loss)

            if self.args.n_gpu > 1:
                all_loss = all_loss.mean()
        return all_loss.detach(), outputs.logits, inputs.labels

In [18]:
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

<ipython-input-18-4f3ad2d139be>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [20]:
batch_size = 4
steps = len(valid_ds) // batch_size
steps

561

In [21]:
training_args = TrainingArguments(
    output_dir="./wav2vec_large_st_1",
    save_total_limit=1,

    group_by_length=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="steps",
    # fp16=True,
    gradient_checkpointing=True,
    learning_rate=1e-4,
    warmup_steps=20,

    save_steps=steps,
    eval_steps=steps,
    logging_steps=steps,
    max_steps=steps*5,

    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False
)

In [25]:
trainer = CTCTrainer(
    model=model,
    data_collator=partial(custom_collate, processor),
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=valid_ds,
    eval_dataset=valid_ds,
    tokenizer=processor.feature_extractor
)

In [26]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
from pyctcdecode import build_ctcdecoder
from transformers import Wav2Vec2ProcessorWithLM
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}
decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="./4gram_correct.arpa",
)
processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=extractor,
    tokenizer=tokenizer,
    decoder=decoder
)

In [ ]:
import os
test_ds = Wav2VecDataset("./dataset/test/public_test", os.listdir("./dataset/test/public_test"))
len(test_ds)

In [ ]:
def test_collate(processor, batch):
    processed_batch = [
        processor(i["input_values"], sampling_rate=i["sample_rate"]) for i in batch
    ]
    input_features = [{"input_values": i.input_values[0]} for i in processed_batch]
    input_features = processor.pad(input_features, padding=True, return_tensors="pt")
    input_features["id"] = [i["file"] for i in batch]
    return input_features

In [ ]:
test_loader = DataLoader(test_ds, shuffle=False, batch_size=16, collate_fn=partial(test_collate, processor))
len(test_loader)

In [ ]:
def wav2vec_inference(model, test_loader, processor, device=None):
    if device is None:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.eval()
    model = model.to(device)
    pred_sentences = {}
    for idx, batch in enumerate(test_loader, 1):
        X_test =  batch["input_values"].to(device)
        X_test = X_test.half()
        file_test = batch["id"]
        with torch.set_grad_enabled(False):
            logits = model(input_values=X_test).logits
        logits = torch.argmax(logits, dim=-1)
        transcriptions = processor.batch_decode(logits, skip_special_tokens=True)
        for file_id, trans in zip(file_test, transcriptions):
            pred_sentences[file_id] = trans
        # pred_sentences += transcriptions
        print("\r", end="")
        print(f"\r {idx} / {len(test_loader)}", end = "" if idx != len(test_loader) else "\n")
    return pred_sentences

In [ ]:
def wav2vec_lm_inference(model, test_loader, processor_with_lm, device=None):
    if device is None:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.eval()
    model = model.to(device)
    pred_sentences = {}
    for idx, batch in enumerate(test_loader, 1):
        X_test =  batch["input_values"].to(device)
        file_test = batch["id"]
        with torch.set_grad_enabled(False):
            logits = model(input_values=X_test).logits
        # logits = torch.argmax(logits, dim=-1)
        logits = logits.cpu().clone().detach().numpy()
        # transcriptions = processor.batch_decode(logits, skip_special_tokens=True)
        transcriptions = processor_with_lm.batch_decode(logits, beam_width=64).text
        for file_id, trans in zip(file_test, transcriptions):
            pred_sentences[file_id] = trans
        # pred_sentences += transcriptions
        print("\r", end="")
        print(f"\r {idx} / {len(test_loader)}", end = "" if idx != len(test_loader) else "\n")
    return pred_sentences

In [ ]:
pred_sens = wav2vec_inference(model, test_loader, processor, torch.device("cuda:0"))

In [ ]:
pred_sens = wav2vec_lm_inference(model, test_loader, processor_with_lm, torch.device("cuda:0"))

In [ ]:
with open("./4ram_test_sentences.json", "w", encoding="utf-8") as f:
    json.dump(pred_sens, f, ensure_ascii=False)
    f.close()